# Working with Decision Trees

A **decision tree** is a classification algorithm where a series of true/false questions about the data are answered to predict the value of a target variable. This usually is visualized by a tree that one traces to make predictions. A nice feature of this algorithm is that it's a heuristic a human can easily interpret and use. However, decision trees are especially prone to overfitting.

Decision tree classifiers can be implemented using the **scikit-learn** class `DecisionTreeClassifier`. The algorithm tries to train a decision tree that quickly makes accurate decisions on training data.

The hyperparameter I want to draw particular attention to is the maximum depth a decision tree may have. Trees with high depth may be more prone to overfitting with trees with low depth, while trees with low depth may underfit.

We will see how well decision trees predict who survived the *Titanic* disaster. I load in the *Titanic* dataset below.

In [25]:
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report

In [26]:
titanic = pd.read_csv("titanic.csv")
titanic.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [3]:
titanic_train, titanic_test = train_test_split(titanic)
titanic_train.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
17,1,2,Mr. Charles Eugene Williams,male,23.0,0,0,13.0000
168,0,3,Mr. Lee Ling,male,28.0,0,0,56.4958
264,0,3,Mr. Ernesti Arvid Panula,male,16.0,4,1,39.6875
365,1,3,Mrs. (Mantoura Boulos) Moussa,female,35.0,0,0,7.2292
755,1,1,the Countess. of (Lucy Noel Martha Dyer-Edward...,female,33.0,0,0,86.5000


## Fitting a Decision Tree

We will fit a decision tree without specifying a maximum depth. We will also visualize the tree. (I grabbed the code for visualizing the tree from a [blog post by "Russel"](https://medium.com/@rnbrown/creating-and-visualizing-decision-trees-with-python-f8e8fa394176).)

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

In [28]:
tree1 = DecisionTreeClassifier()

tree1 = tree1.fit(X=titanic_train.replace({'Sex': {'male': 0, 'female': 1}}    # Replace strings with numbers
                                         ).drop(["Survived", "Name"], axis=1),
                  y=titanic_train.Survived)

# Example prediction
tree1.predict([[2, 0, 26, 0, 0, 30]])    # A male in second class age 26 with no spouse or child aboard who paid $30 fare

array([0], dtype=int64)

In [29]:
pred1 = tree1.predict(titanic_train.replace({'Sex': {'male': 0, 'female': 1}}
                                           ).drop(["Survived", "Name"], axis=1))
print(classification_report(titanic_train.Survived, pred1))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99       410
          1       1.00      0.97      0.98       255

avg / total       0.99      0.99      0.99       665



We can see that on the training data the algorithm is highly accurate, but there's a good chance the model overfit the data.

We can visualize the resulting tree like so. (You will need to install [Graphviz](http://www.graphviz.org/), an open source software package for visualizing graphs, including decision trees.)

In [30]:
# From here: https://medium.com/@rnbrown/creating-and-visualizing-decision-trees-with-python-f8e8fa394176
dot_data = StringIO()

export_graphviz(tree1,    # Function for exporting a visualization of the tree
                out_file=dot_data,
                # Data controlling the display of the graph
                filled=True, rounded=True,
                special_characters=True,
                feature_names=["Pclass", "Sex", "Age", "Siblings/Spouses Aboard", "Parents/Children Aboard",
                               "Fare"],    # Use the name of the features
                proportion=True)    # Show proportions for labels

# Display graph in Jupyter notebook
graph1 = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph1.create_png())

InvocationException: GraphViz's executables not found

A graph this complex is probably overfitting. In fact, let's peek to see how this would do on the test data.

In [31]:
pred2 = tree1.predict(titanic_test.replace({'Sex': {'male': 0, 'female': 1}}
                                          ).drop(["Survived", "Name"], axis=1))
print(classification_report(titanic_test.Survived, pred2))

             precision    recall  f1-score   support

          0       0.82      0.79      0.80       135
          1       0.68      0.72      0.70        87

avg / total       0.76      0.76      0.76       222



Performance dropped significantly. This is likely no better than the "predict most frequent label" algorithm.

## Restricting Tree Depth

We can control overfitting by restricting the depth of the tree. For example, let's see a tree that does not go deeper than three levels.

In [32]:
tree2 = DecisionTreeClassifier(max_depth=3)

tree2 = tree2.fit(X=titanic_train.replace({'Sex': {'male': 0, 'female': 1}}    # Replace strings with numbers
                                         ).drop(["Survived", "Name"], axis=1),
                  y=titanic_train.Survived)

dot_data = StringIO()

export_graphviz(tree2,    # Function for exporting a visualization of the tree
                out_file=dot_data,
                # Data controlling the display of the graph
                filled=True, rounded=True,
                special_characters=True,
                feature_names=["Pclass", "Sex", "Age", "Siblings/Spouses Aboard", "Parents/Children Aboard",
                               "Fare"],
                proportion=True)

# Display graph in Jupyter notebook
graph2 = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph2.create_png())

InvocationException: GraphViz's executables not found

Now let's use cross-validation to decide the appropriate tree depth.

In [33]:
m_candidate = [2, 3, 4, 5, 6, 7, 8, 9, 10]    # Candidate depths
res = dict()

for m in m_candidate:
    pred3 = DecisionTreeClassifier(max_depth=m)
    res[m] = cross_validate(pred3,
                            X=titanic_train.replace({'Sex': {'male': 0, 'female': 1}}    # Replace strings with numbers
                                         ).drop(["Survived", "Name"], axis=1),
                            y=titanic_train.Survived,
                            cv=10,
                            return_train_score=False,
                            scoring='accuracy')

resdf = DataFrame({(i, j): res[i][j]
                             for i in res.keys()
                             for j in res[i].keys()}).T

resdf.loc[(slice(None), 'test_score'), :]

,,0,1,2,3,4,5,6,7,8,9
2,test_score,0.746269,0.731343,0.820896,0.761194,0.761194,0.712121,0.772727,0.772727,0.863636,0.803030
3,test_score,0.820896,0.820896,0.835821,0.776119,0.805970,0.757576,0.833333,0.848485,0.878788,0.818182
4,test_score,0.835821,0.805970,0.805970,0.776119,0.805970,0.787879,0.863636,0.818182,0.893939,0.803030
5,test_score,0.761194,0.805970,0.820896,0.761194,0.835821,0.757576,0.863636,0.818182,0.893939,0.787879
6,test_score,0.776119,0.791045,0.820896,0.746269,0.865672,0.772727,0.863636,0.787879,0.848485,0.818182
7,test_score,0.731343,0.776119,0.820896,0.761194,0.820896,0.757576,0.848485,0.757576,0.848485,0.863636
8,test_score,0.731343,0.805970,0.835821,0.776119,0.880597,0.757576,0.833333,0.757576,0.848485,0.833333
9,test_score,0.686567,0.805970,0.835821,0.761194,0.820896,0.757576,0.863636,0.803030,0.878788,0.818182
10,test_score,0.776119,0.776119,0.805970,0.776119,0.850746,0.787879,0.833333,0.787879,0.878788,0.833333


In [34]:
resdf.loc[(slice(None), 'test_score'), :].mean(axis=1)

2   test_score    0.774514
3   test_score    0.819607
4   test_score    0.819652
5   test_score    0.810629
6   test_score    0.809091
7   test_score    0.798621
8   test_score    0.806015
9   test_score    0.803166
10  test_score    0.810629
dtype: float64

Maximum predictive accuracy occurs when the maximum depth is 4. Let's see the final result.

In [35]:
tree4 = DecisionTreeClassifier(max_depth=4)

tree4 = tree4.fit(X=titanic_train.replace({'Sex': {'male': 0, 'female': 1}}    # Replace strings with numbers
                                         ).drop(["Survived", "Name"], axis=1),
                  y=titanic_train.Survived)

dot_data = StringIO()

export_graphviz(tree4,    # Function for exporting a visualization of the tree
                out_file=dot_data,
                # Data controlling the display of the graph
                filled=True, rounded=True,
                special_characters=True,
                feature_names=["Pclass", "Sex", "Age", "Siblings/Spouses Aboard", "Parents/Children Aboard",
                               "Fare"],
                proportion=True)

# Display graph in Jupyter notebook
graph3 = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph3.create_png())

InvocationException: GraphViz's executables not found

In [36]:
survived_test_predict = tree4.predict(X=titanic_test.replace(
    {'Sex': {'male': 0, 'female': 1}}
).drop(["Survived", "Name"], axis=1))

In [37]:
print(classification_report(titanic_test.Survived, survived_test_predict))

             precision    recall  f1-score   support

          0       0.85      0.90      0.87       135
          1       0.83      0.75      0.79        87

avg / total       0.84      0.84      0.84       222



The metrics of the decision tree look good. These are better than when we allowed the tree to have any depth.